In [265]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input,Dense,Flatten
from keras.layers import LSTM
from numpy import array

In [266]:
df_train = pd.read_csv('./Train_X.csv',encoding= 'cp950')
df_test = pd.read_csv('./Test_X.csv',encoding= 'cp950')

In [267]:
df_train = df_train.sort_values('日期')
df_test = df_test.sort_values('日期')

stock_list_train = (df['股票代號'].unique()).tolist()
stock_list_test = (df['股票代號'].unique()).tolist()
data_train = {}
data_test ={}
for name in stock_list:
    data_train[name]=df_train[df_train['股票代號']==name].loc[:,['開盤價', '最高價','最低價','收盤價','成交量.股.']]
    data_test[name]=df_test[df_test['股票代號']==name].loc[:,['開盤價', '最高價','最低價','收盤價','成交量.股.']]

In [268]:
#定義正規化函式
def normalize(train):
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

#定義餵入的feature跟label序列
def train_(df, ref_day=10, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]['收盤價']))
    return np.array(X_train), np.array(Y_train)

TWA00_train= normalize(data_train['TWA00'])
TWA00_test= normalize(data_test['TWA00'])

In [269]:
X_train,Y_train=train_(TWA00_train,7,1)
X_test,Y_test=train_(TWA00_test,7,1)

#split_boundary = int(X.shape[0] * 0.9)
train_x = X[: split_boundary]
test_x = X[split_boundary:]
train_y = Y[: split_boundary]
test_y = Y[split_boundary:]

In [278]:
X_train.shape[2]

5

In [270]:
# define model
units = 50
inputs1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
lstm1=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(inputs1)
lstm2=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(lstm1)
lstm3=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(lstm2)
final=LSTM(1, activation='tanh', recurrent_activation='hard_sigmoid',return_sequences = False)(lstm3)
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.99, beta_2=0.999, amsgrad=False)

model = Model(inputs=inputs1, outputs=final)
model.compile(optimizer=adam, loss='mean_squared_error')


In [271]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 20)

Epoch 1/100
229/229 [==============================] - 17s 75ms/step - loss: 0.0270
Epoch 2/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0149
Epoch 3/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0114
Epoch 4/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0104
Epoch 5/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0096
Epoch 6/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 7/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0077
Epoch 8/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0073
Epoch 9/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0073
Epoch 10/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 11/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 12/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0051
Epoch 13/10

229/229 [==============================] - 0s 2ms/step - loss: 0.0011
Epoch 100/100
229/229 [==============================] - 0s 2ms/step - loss: 0.0011


In [274]:
predict_y = model.predict(X_test)

In [279]:
#model.evaluate(X_test,Y_test)
X_test

array([[[-0.54779716, -0.57050352, -0.58669587, -0.64190359,
          0.15542402],
        [-0.36900756, -0.34236033, -0.38896117, -0.34535018,
          0.47063377],
        [-0.23810572, -0.17874068, -0.20535224, -0.11112965,
          0.20256904],
        [-0.00161783, -0.08094679, -0.02542377, -0.10448179,
          0.03353686],
        [-0.07228463,  0.04169135, -0.01751853,  0.11254262,
          0.30494918],
        [ 0.16158884,  0.07197426,  0.1309704 ,  0.04513185,
         -0.16387352],
        [ 0.12296801,  0.11290471,  0.1429449 ,  0.18095916,
         -0.28123302]],

       [[-0.36900756, -0.34236033, -0.38896117, -0.34535018,
          0.47063377],
        [-0.23810572, -0.17874068, -0.20535224, -0.11112965,
          0.20256904],
        [-0.00161783, -0.08094679, -0.02542377, -0.10448179,
          0.03353686],
        [-0.07228463,  0.04169135, -0.01751853,  0.11254262,
          0.30494918],
        [ 0.16158884,  0.07197426,  0.1309704 ,  0.04513185,
         -0.1

In [ ]:
def denormalize(train):
    denorm = train*(np.max(data_train['TWA00']['收盤價'])-np.min(data_train['TWA00']['收盤價']))+np.mean(data_train['TWA00']['收盤價'])
    return denorm

In [ ]:
Y_test = pd.DataFrame(Y_test)
test_y = denormalize(Y_test)
z = denormalize(predict_y)

In [ ]:
test_y

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(test_y, color = 'red', label = 'Real Price')  # 紅線表示真實股價
plt.plot(z, color = 'blue', label = 'Predicted Price')  # 藍線表示預測股價
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()